In [1]:
import os
import config

import pandas as pd
import numpy as np
from scipy.stats import beta

from flask import Flask, jsonify, render_template

import mysql.connector

from datetime import datetime, date, time, timedelta
from time import sleep
import pendulum

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import requests

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

In [2]:
import os, time
time.strftime('%X %x %Z')

os.environ['TZ'] = 'Europe/London'
time.tzset()

time.strftime('%X %x %Z')

'14:22:03 08/30/19 BST'

In [3]:
# Fetch the service account key JSON file contents
cred = credentials.Certificate(config.firebase_file_path)
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': config.databaseURL
})

In [12]:
from pytz import timezone

orders_db = "orders.json"
r = requests.get(config.databaseURL + orders_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    orders_df = pd.DataFrame.from_dict(data, orient="columns")
print(orders_df)
orders_df = orders_df.astype({"orderID": int})

orderShipped_db = "order_shipped.json"
r = requests.get(config.databaseURL + orderShipped_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    orders_shipped_df = pd.DataFrame.from_dict(data, orient="columns")
print(orders_shipped_df)
orders_shipped_df = orders_shipped_df.astype({"orderID": int})
orders_shipped_df.drop_duplicates('orderID',keep='first',inplace=True)

order_2 = pd.merge(orders_df, orders_shipped_df, on="orderID", how="outer")
print(order_2)
buyers_db = "buyers.json"
r = requests.get(config.databaseURL + buyers_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    buyers_df = pd.DataFrame.from_dict(data, orient="columns")

buyers_df = buyers_df.astype({"orderID": int})

buyers_df["lastInitial"] = buyers_df["lastName"].astype(str).str[0]
buyers_df["firstThree"] = buyers_df["firstName"].astype(str).str[:3]

buyers_df["name"] = buyers_df["firstThree"] + " " + buyers_df["lastInitial"]

buyers_df = buyers_df[["name", "orderID"]]

open_orders = pd.merge(buyers_df, order_2, on="orderID", how="inner")

open_orders['orderDate'] = pd.to_datetime(open_orders['orderDate'])

# open_orders['shipByDate'] = pd.to_datetime(open_orders['shipByDate'])

open_orders_df = open_orders.loc[(open_orders["dateShipped"]=="nan")|(open_orders['dateShipped'].isnull()),:]
print(open_orders_df)
# open_orders_df['processing_time'] = open_orders_df['shipByDate_local'].sub(open_orders_df['orderDate_local'], axis=0)

now = datetime.now()
now = timezone('US/Central').localize(now)

open_orders_df['now'] = now

for index, item in open_orders_df.iterrows():
    
    shipByDate = datetime.strptime(item.shipByDate, "%Y-%m-%d")
    shipByDate = timezone('US/Central').localize(shipByDate)

    shipByDate =shipByDate + timedelta(hours=22)

    open_orders_df.loc[index,'shipByDate_local']=shipByDate

           buyerID   orderDate     orderID orderTime  \
0         zvq52mlo  2019-04-19  1441612324     12:04   
1     sshelburne12  2019-05-15  1461258703      6:36   
2    marissaspacey  2019-06-17  1465581068     10:00   
3          lmcq741  2019-06-21  1467289834     18:01   
4       admoorhead  2019-07-02  1471406642     22:36   
5          cjoy123  2019-07-04  1471997258     16:03   
6      gauklerin09  2019-07-23  1479218984     21:31   
7         dick0390  2019-07-09  1482734683     10:52   
8   jessicaholder4  2019-07-20  1487089393     16:19   
9                   2019-07-26  1480128610     11:33   
10         Fudge69  2019-07-27  1480472568     10:57   
11   brittanyb0710  2019-08-07  1484941268     13:42   
12     HeatherW290  2019-08-05  1493211993     11:03   
13         cjoy123  2019-08-10  1486172004     14:33   
14      ivesterh89  2019-08-01  1482480540       nan   
15         laaia22  2019-07-29  1481079872       nan   
16      glyndasue5  2019-07-28  1481051598      

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
open_orders_df

,name,orderID,buyerID,orderDate,orderTime,orderURL,record_date,shipByDate,dateShipped,now,shipByDate_local
19,Jes H,1491061832,jessicaholder4,2019-08-22,11:54,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-08-23,nan,2019-08-30 14:25:46.180025-05:00,2019-08-23 22:00:00-05:00
20,Mer N,1493294910,sharpys,2019-08-28,23:11,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-09-03,nan,2019-08-30 14:25:46.180025-05:00,2019-09-03 22:00:00-05:00
22,Dan T,1493707488,dftipton,2019-08-29,01:38,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-09-09,nan,2019-08-30 14:25:46.180025-05:00,2019-09-09 22:00:00-05:00


In [10]:
for index, item in open_orders_df.iterrows():
    timeLeft2=(item.shipByDate_local-item.now).total_seconds()
    d = timeLeft2//86400
    d = int(d)
    print(d)
    h = (timeLeft2%86400)//3600
    h =int(h)
    if d >= 2:
        d_text='days'
    else:
        d_text='day'
    if h >= 2:
        h_text ='hours'
    else:
        h_text='hour'
    
    open_orders_df.loc[index,'timeLeft']=f"{d} {d_text} {h} {h_text}"
open_orders_df = open_orders_df.sort_values(by="shipByDate", ascending=True)

open_orders_df

-7
4
10


,name,orderID,buyerID,orderDate,orderTime,orderURL,record_date,shipByDate,dateShipped,now,shipByDate_local,timeLeft
19,Jes H,1491061832,jessicaholder4,2019-08-22,11:54,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-08-23,nan,2019-08-30 14:25:46.180025-05:00,2019-08-23 22:00:00-05:00,-7 day 7 hours
20,Mer N,1493294910,sharpys,2019-08-28,23:11,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-09-03,nan,2019-08-30 14:25:46.180025-05:00,2019-09-03 22:00:00-05:00,4 days 7 hours
22,Dan T,1493707488,dftipton,2019-08-29,01:38,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-09-09,nan,2019-08-30 14:25:46.180025-05:00,2019-09-09 22:00:00-05:00,10 days 7 hours


In [7]:
open_orders = []

for index, item in open_orders_df.iterrows():
    openOrderDict={}
    openOrderDict['abuyerName']=item[0]
    openOrderDict['borderDate']=item[3].strftime('%m/%d/%y')
    openOrderDict['cshipByDate']=item[10].strftime('%m/%d/%y')
    openOrderDict['dtimeLeft']=str(item[-1])
    openOrderDict['eorderURL']=str(item[5])
    open_orders.append(openOrderDict)

open_orders

[{'abuyerName': 'Jes H',
  'borderDate': '08/22/19',
  'cshipByDate': '08/23/19',
  'dtimeLeft': '-7 day 7 hours',
  'eorderURL': 'https://www.etsy.com/your/orders/sold?ref=seller-platform-mcnav&order_id=1491061832'},
 {'abuyerName': 'Mer N',
  'borderDate': '08/28/19',
  'cshipByDate': '09/03/19',
  'dtimeLeft': '4 days 7 hours',
  'eorderURL': 'https://www.etsy.com/your/orders/sold?ref=seller-platform-mcnav&order_id=1493294910'},
 {'abuyerName': 'Dan T',
  'borderDate': '08/29/19',
  'cshipByDate': '09/09/19',
  'dtimeLeft': '10 days 7 hours',
  'eorderURL': 'https://www.etsy.com/your/orders/sold?ref=seller-platform-mcnav&order_id=1493707488'}]